# Average Headway Nationaltheateret
Unsuccessful approach of using trips on a day to find the average headway of platforms at Nationaltheateret.

In [ ]:
# Installing packages for jupyter lab
%pip install -r requirements.txt 

In [26]:
import pandas as pd

In [27]:
nodes_oslo = pd.read_csv("../../Extraction/out/nodes_GTFS_OSLO.csv")
edges_oslo = pd.read_csv("../../Extraction/out/edges_GTFS_OSLO_with_mondayTrips.csv")
display(nodes_oslo.head()) 
display(edges_oslo.head())

,id,stopPlaceId,name,lat,lon,modes,stopType
0,NSR:Quay:99688,NSR:Quay:99688,Vækerø,59.915650,10.651833,"bus,coach service",multimodal
1,NSR:Quay:11880,NSR:Quay:11880,Maritim,59.918496,10.667065,"bus,coach service",multimodal
2,NSR:Quay:11829,NSR:Quay:11829,Sjølyst,59.920456,10.676683,"bus,coach service",multimodal
3,NSR:Quay:7700,NSR:Quay:7700,Vika atrium,59.910762,10.722139,"bus,coach service",multimodal
4,NSR:Quay:11969,NSR:Quay:11969,Oslo bussterminal,59.911940,10.756710,bus,bus


,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
0,NSR:Quay:99688,NSR:Quay:11880,BRA:Line:4_6169,169,bus,Brakar,60.0,7,14
1,NSR:Quay:11880,NSR:Quay:11829,BRA:Line:4_6169,169,bus,Brakar,60.0,7,14
2,NSR:Quay:11829,NSR:Quay:7700,BRA:Line:4_6169,169,bus,Brakar,120.0,7,14
3,NSR:Quay:7700,NSR:Quay:11969,BRA:Line:4_6169,169,bus,Brakar,660.0,7,14
4,NSR:Quay:12002,NSR:Quay:7699,BRA:Line:4_6169,169,bus,Brakar,660.0,7,14


In [28]:
edges_trains_oslo = edges_oslo[edges_oslo["mode"] == "rail"]
nodes_trains_oslo = nodes_oslo[nodes_oslo["modes"].str.contains("rail")]
display(edges_trains_oslo.head())
display(nodes_trains_oslo.head())
print(f"Edges shape Oslo: {edges_trains_oslo.shape} | Nodes/Stops shape: {nodes_trains_oslo.shape}")

,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
18,NSR:Quay:236,NSR:Quay:476,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
19,NSR:Quay:476,NSR:Quay:557,FLT:Line:FLY2,FLY2,rail,Flytoget,360.0,2940,100
20,NSR:Quay:570,NSR:Quay:475,FLT:Line:FLY2,FLY2,rail,Flytoget,180.0,2916,100
21,NSR:Quay:475,NSR:Quay:239,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2916,100
22,NSR:Quay:570,NSR:Quay:239,FLT:Line:FLY2,FLY2,rail,Flytoget,420.0,24,100


,id,stopPlaceId,name,lat,lon,modes,stopType
12,NSR:Quay:236,NSR:Quay:236,Skøyen stasjon,59.922268,10.678630,rail,rail
13,NSR:Quay:476,NSR:Quay:476,Nationaltheatret stasjon,59.915260,10.727513,rail,rail
14,NSR:Quay:557,NSR:Quay:557,Oslo S,59.910261,10.754866,rail,rail
15,NSR:Quay:570,NSR:Quay:570,Oslo S,59.911022,10.755594,rail,rail
16,NSR:Quay:475,NSR:Quay:475,Nationaltheatret stasjon,59.915537,10.728612,rail,rail


Edges shape Oslo: (194, 9) | Nodes/Stops shape: (61, 7)


In [29]:
platformer_national = []

for row in nodes_trains_oslo.itertuples():
    if row.name == "Nationaltheatret stasjon":
        platformer_national.append(row.id)

print(platformer_national)

['NSR:Quay:476', 'NSR:Quay:475', 'NSR:Quay:478', 'NSR:Quay:477']


In [30]:
to_from_national = edges_trains_oslo[
    (edges_trains_oslo["from"].isin(platformer_national)) |
    (edges_trains_oslo["to"].isin(platformer_national))
    ]

In [31]:
display(to_from_national.sort_values(by=["from", "to", "tripsOn2025_11_17"]).head())

,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
4756,NSR:Quay:236,NSR:Quay:476,VYG:Line:R12,R12,rail,Vy,180.0,16,40
4745,NSR:Quay:236,NSR:Quay:476,VYG:Line:RE10,RE10,rail,Vy,180.0,5,41
4614,NSR:Quay:236,NSR:Quay:476,VYG:Line:R14,R14,rail,Vy,180.0,5,42
18,NSR:Quay:236,NSR:Quay:476,FLT:Line:FLY2,FLY2,rail,Flytoget,240.0,2940,100
23,NSR:Quay:236,NSR:Quay:476,FLT:Line:FLY1,FLY1,rail,Flytoget,240.0,9205,118


In [32]:
to_from_national = to_from_national.drop(columns=["lineId", "lineCode", "authority"])

to_from_national = (
    to_from_national.groupby(["from", "to"], as_index=False)
       .agg({
           "mode": "first",
           "travelTimeSec": "first",
           "tripsInFeed": "first",
           "tripsOn2025_11_17": "sum"
       })
)

In [33]:
display(to_from_national)

,from,to,mode,travelTimeSec,tripsInFeed,tripsOn2025_11_17
0,NSR:Quay:236,NSR:Quay:476,rail,240.0,2940,1077
1,NSR:Quay:236,NSR:Quay:477,rail,180.0,57,1131
2,NSR:Quay:237,NSR:Quay:477,rail,180.0,8,77
3,NSR:Quay:238,NSR:Quay:476,rail,180.0,74,1005
4,NSR:Quay:238,NSR:Quay:477,rail,180.0,6,1007
5,NSR:Quay:475,NSR:Quay:237,rail,180.0,36,965
6,NSR:Quay:475,NSR:Quay:239,rail,240.0,2916,1456
7,NSR:Quay:476,NSR:Quay:551,rail,180.0,4,1015
8,NSR:Quay:476,NSR:Quay:555,rail,180.0,216,889
9,NSR:Quay:476,NSR:Quay:557,rail,360.0,2940,218


In [34]:
platforms = {}

for platform in platformer_national:
    platforms[platform] = to_from_national[(to_from_national["from"] == platform) | (to_from_national["to"] == platform)]

In [35]:
display(platforms["NSR:Quay:478"])

,from,to,mode,travelTimeSec,tripsInFeed,tripsOn2025_11_17
18,NSR:Quay:478,NSR:Quay:237,rail,180.0,2,1166
19,NSR:Quay:478,NSR:Quay:239,rail,150.0,2,1238
21,NSR:Quay:550,NSR:Quay:478,rail,60.0,1,777
23,NSR:Quay:553,NSR:Quay:478,rail,60.0,2,931
25,NSR:Quay:554,NSR:Quay:478,rail,121.0,2,82
27,NSR:Quay:559,NSR:Quay:478,rail,60.0,65,242
29,NSR:Quay:561,NSR:Quay:478,rail,60.0,43,965
30,NSR:Quay:566,NSR:Quay:478,rail,60.0,2,10


In [36]:
platform_trip_counts = {}

for platform, df in platforms.items():

    outgoing = df.loc[df["from"] == platform, "tripsOn2025_11_17"].sum()

    incoming = df.loc[df["to"] == platform, "tripsOn2025_11_17"].sum()

    platform_trip_counts[platform] = {
        "incoming": int(incoming),
        "outgoing": int(outgoing),
    }

platform_trip_counts


{'NSR:Quay:476': {'incoming': 2082, 'outgoing': 2437},
 'NSR:Quay:475': {'incoming': 3215, 'outgoing': 2421},
 'NSR:Quay:478': {'incoming': 3007, 'outgoing': 2404},
 'NSR:Quay:477': {'incoming': 2215, 'outgoing': 1562}}

In [37]:
df_counts = pd.DataFrame.from_dict(platform_trip_counts, orient="index")

df_counts = df_counts.reset_index().rename(columns={"index": "platform"})

df_counts["max_in_out"] = df_counts[["incoming", "outgoing"]].max(axis=1)

df_counts["avg_headway_sec"] = (24*60*60)/df_counts["max_in_out"]

df_counts

,platform,incoming,outgoing,max_in_out,avg_headway_sec
0,NSR:Quay:476,2082,2437,2437,35.453426
1,NSR:Quay:475,3215,2421,3215,26.874028
2,NSR:Quay:478,3007,2404,3007,28.732956
3,NSR:Quay:477,2215,1562,2215,39.006772
